In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp

In [4]:
expected_returns = pd.read_excel(r"E:\大三\大三上\Optimization in Finance\Final Project\Portfolio\Half Year\Use for Code.xlsx", header=None).values
cov_matrix = pd.read_excel(r"E:\大三\大三上\Optimization in Finance\Final Project\Portfolio\Covariance\Covariance.xlsx", header=None).values  # 确保是 N×N

min_weight = 0.01  # 最小持仓 1%
M = 1.0            # 大数上界
max_risk_std = 0.1  
max_risk_var = max_risk_std ** 2


In [5]:
expected_returns.shape

(3, 499)

In [6]:
for i in range(3):
    expected_return = expected_returns[i, :].flatten()  # 第 i 期的预期收益向量
    N = len(expected_return)
    print(f"股票数量: {N}")
    assert cov_matrix.shape == (N, N), "协方差矩阵维度不匹配！"

    w = cp.Variable(N)          # 连续权重
    z = cp.Variable(N, boolean=True)  # 0-1 变量：是否持仓

    # 目标：最大化组合预期收益 ===
    objective = cp.Maximize(expected_return @ w)
    constraints = [
        cp.sum(w) == 1.0,        # 权重和为 1（满仓）
        w >= 0,
        w <= 0.2,                # 单资产上限 20%
        cp.sum(z) <= 30,         # 最多持 30 只
        w >= min_weight * z,     # 持仓则 ≥ min_weight
        w <= M * z,              # 不持仓则 = 0
        cp.quad_form(w, cov_matrix) <= max_risk_var,  
    ]

    # === 求解 ===
    prob = cp.Problem(objective, constraints)

    print(f"正在求解第 {i+1} 期：最大化收益（风险上限 {max_risk_std*100:.2f}%）...")
    try:
        prob.solve(solver=cp.ECOS_BB, verbose=False, max_iters=500)
    except Exception as e:
        print("ECOS_BB 求解失败:", e)
        raise

    if w.value is not None and prob.status in ["optimal", "optimal_inaccurate"]:
        weights = w.value
        weights = np.maximum(weights, 0)
        weights = weights / np.sum(weights)  # 数值归一化

        portfolio_risk = np.sqrt(weights @ cov_matrix @ weights)
        portfolio_return = expected_return @ weights  # 注意：用当前期的 expected_return！

        rf = 0.005
        sharpe_ratio = (portfolio_return - rf) / portfolio_risk

        print(f"\n 第 {i+1} 期结果:")
        print(f"组合风险（标准差）: {portfolio_risk:.6f} ({portfolio_risk*100:.3f}%)")
        print(f"组合预期收益: {portfolio_return:.6f} ({portfolio_return*100:.3f}%)")
        print(f"夏普比率: {sharpe_ratio:.4f}")  
        print(f"持仓股票数量: {np.sum(weights > 1e-4)} / {N}")

        nonzero_indices = np.where(weights > 1e-4)[0]
        print(f"\n 非零持仓（共 {len(nonzero_indices)} 只）:")
        ''' Output the Portfolio
        for idx in nonzero_indices:
            print(f"  股票 {idx+1:3d}: {weights[idx]:.4f} ({weights[idx]*100:6.2f}%)")
        '''
        result_df = pd.DataFrame({
            "Stock_ID": [f"Stock_{j+1}" for j in range(N)],
            "Weight": weights
        })
        file_name = f"Season{i+1}.csv"
        #result_df.to_csv(file_name, index=False)
        print(f"\n 权重已保存至 {file_name}")

    else:
        print(f" 第 {i+1} 期优化失败！")
        print("状态:", prob.status)
        print("可能原因：风险上限太严、无可行解、求解器能力不足")
        

股票数量: 499
正在求解第 1 期：最大化收益（风险上限 10.00%）...

 第 1 期结果:
组合风险（标准差）: 0.055906 (5.591%)
组合预期收益: 0.100000 (10.000%)
夏普比率: 1.6993
持仓股票数量: 30 / 499

 非零持仓（共 30 只）:

 权重已保存至 Season1.csv
股票数量: 499
正在求解第 2 期：最大化收益（风险上限 10.00%）...

 第 2 期结果:
组合风险（标准差）: 0.096887 (9.689%)
组合预期收益: 0.100000 (10.000%)
夏普比率: 0.9805
持仓股票数量: 6 / 499

 非零持仓（共 6 只）:

 权重已保存至 Season2.csv
股票数量: 499
正在求解第 3 期：最大化收益（风险上限 10.00%）...

 第 3 期结果:
组合风险（标准差）: 0.054202 (5.420%)
组合预期收益: 0.100000 (10.000%)
夏普比率: 1.7527
持仓股票数量: 5 / 499

 非零持仓（共 5 只）:

 权重已保存至 Season3.csv
